##### L'objectif est de calculer des taux 

In [45]:
from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *


## Adaptation pour faciliter l'usage de ce notebook

_Ce correctif permet de redéfinir plusieurs fois la même variable sans provoquer d'erreur._

# Système socio-fiscal

In [46]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

# Réforme

In [47]:
from openfisca_core import reforms

In [4]:
Reform = reforms.make_reform(
    key = 'cotisations_revenu_de_base',
    name = u"Réforme des cotisations pour un Revenu de base",
    reference = tax_benefit_system,
    )

In [5]:
class cotisations_contributives(Reform.Variable):
    column = FloatCol
    entity_class = Individus
    label = u"Nouvelles cotisations contributives"


    def function(self, simulation, period):
        ags = simulation.calculate('ags', period)
        agff_tranche_a_employeur = simulation.calculate('agff_employeur', period)
        apec_employeur = simulation.calculate('apec_employeur', period)
        arrco_tranche_a_employeur = simulation.calculate('arrco_employeur', period)
        assedic_employeur = simulation.calculate('chomage_employeur', period)
        cotisation_exceptionnelle_temporaire_employeur = simulation.calculate('cotisation_exceptionnelle_temporaire_employeur', period)
        fonds_emploi_hospitalier = simulation.calculate('fonds_emploi_hospitalier', period)
        ircantec_employeur = simulation.calculate('ircantec_employeur', period)
        pension_civile_employeur = simulation.calculate('pension_civile_employeur', period)
        prevoyance_obligatoire_cadre = simulation.calculate('prevoyance_obligatoire_cadre', period)
        rafp_employeur = simulation.calculate('rafp_employeur', period)
        vieillesse_deplafonnee_employeur = simulation.calculate('vieillesse_deplafonnee_employeur', period)
        vieillesse_plafonnee_employeur = simulation.calculate('vieillesse_plafonnee_employeur', period)
        allocations_temporaires_invalidite = simulation.calculate('allocations_temporaires_invalidite', period)
        accident_du_travail = simulation.calculate('accident_du_travail', period)
        agff_tranche_a_employe = simulation.calculate_add('agff_salarie', period)
        agirc_tranche_b_employe = simulation.calculate_add('agirc_salarie', period)
        apec_employe = simulation.calculate_add('apec_salarie', period)
        arrco_tranche_a_employe = simulation.calculate_add('arrco_salarie', period)
        assedic_employe = simulation.calculate_add('chomage_salarie', period)
        cotisation_exceptionnelle_temporaire_employe = simulation.calculate('cotisation_exceptionnelle_temporaire_salarie', period)
        ircantec_employe = simulation.calculate_add('ircantec_salarie', period)
        pension_civile_employe = simulation.calculate_add('pension_civile_salarie', period)
        rafp_employe = simulation.calculate_add('rafp_salarie', period)
        vieillesse_deplafonnee_employe = simulation.calculate_add('vieillesse_deplafonnee_salarie', period)
        vieillesse_plafonnee_employe = simulation.calculate_add('vieillesse_plafonnee_salarie', period)

        cotisations_contributives = (
            # cotisations patronales contributives dans le prive
            ags +
            agff_tranche_a_employeur +
            apec_employeur +
            arrco_tranche_a_employeur +
            #assedic_employeur +
            cotisation_exceptionnelle_temporaire_employeur +
            prevoyance_obligatoire_cadre +  # TODO contributive ou pas
            vieillesse_deplafonnee_employeur +
            vieillesse_plafonnee_employeur +
            # cotisations patronales contributives dans le public
            fonds_emploi_hospitalier +
            ircantec_employeur +
            pension_civile_employeur +
            rafp_employeur +
            # anciennes cot patronales non-contributives classées ici comme contributives
            allocations_temporaires_invalidite +
            accident_du_travail +
            # anciennes cotisations salariales contributives dans le prive
            agff_tranche_a_employe +
            agirc_tranche_b_employe +
            apec_employe +
            arrco_tranche_a_employe +
            assedic_employe +
            cotisation_exceptionnelle_temporaire_employe +
            vieillesse_deplafonnee_employe +
            vieillesse_plafonnee_employe +
            # anciennes cotisations salariales contributives dans le public
            ircantec_employe +
            pension_civile_employe +
            rafp_employe
            )
        return period, cotisations_contributives
        

In [22]:
class nouv_salbrut(Reform.Variable):
    reference = tax_benefit_system.column_by_name['salaire_de_base']

# Le salaire brut se définit dans la réforme comme le salaire super-brut auquel
# on retranche les cotisations contributives 
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        salaire_super_brut = simulation.calculate('salaire_super_brut', period)
        cotisations_contributives = simulation.calculate('cotisations_contributives', period)
        
        nouv_salbrut = (
            salaire_super_brut -
            cotisations_contributives
            )
        return period, nouv_salbrut

In [67]:
class nouv_csg(Reform.Variable):
    reference = tax_benefit_system.column_by_name['csg_imposable_salaire']

# On applique une CSG unique à 15,0% qui finance toutes les prestations non-contributives 
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        nouv_salbrut = simulation.calculate('nouv_salbrut', period)
        
        nouv_csg = (
            -0.15 * nouv_salbrut
            )
        return period, nouv_csg

In [68]:
class salnet(Reform.Variable):
    reference = tax_benefit_system.column_by_name['salaire_net']

# On retire la nouvelle CSG (pas celle qui finance le RDB) pour trouver le nouveau salaire net
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        nouv_salbrut = simulation.calculate('nouv_salbrut', period)
        nouv_csg = simulation.calculate('nouv_csg', period)
        
        salnet = (
            nouv_salbrut +
            nouv_csg
            )
        return period, salnet

In [60]:
class sal(Reform.Variable):
    reference = tax_benefit_system.column_by_name['salaire_de_base']

# Nous sommes partis du nouveau salaire net et par rapport au salaire imposable actuel,
# nous avons supprimé : les heures sup, la déductibilité de CSG
    
    
    def function(self, simulation, period):
        period = period
        salnet = simulation.calculate('salnet', period)
        primes_fonction_publique = simulation.calculate('primes_fonction_publique', period)
        indemnite_residence = simulation.calculate('indemnite_residence', period)
        supp_familial_traitement = simulation.calculate('supp_familial_traitement', period)
        rev_microsocial_declarant1 = simulation.calculate('rev_microsocial_declarant1', period)

        return period, (
            salnet +
            primes_fonction_publique +
            indemnite_residence +
            supp_familial_traitement +
            hsup +
            rev_microsocial_declarant1
            )
    #Pour la période, je retourne la somme de toutes les variables.

# Simulation

In [55]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1975, 1, 1),
        salaire_de_base = 10000,
        ),
    parent2 = dict(
        birth = date(1978, 1, 1),
        salaire_de_base = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [56]:
simulation.calculate("salaire_super_brut")

array([ 9911.64648438,  5911.64648438,     0.        ], dtype=float32)

In [57]:
simulation.calculate("revdisp")

array([ 17205.69140625], dtype=float32)

In [42]:
reform = Reform()

In [43]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1975, 1, 1),
        salaire_de_base = 10000,
        ),
    parent2 = dict(
        birth = date(1978, 1, 1),
        salaire_de_base = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [44]:
reform_simulation.calculate('salaire_super_brut')

array([ 9911.64648438,  5911.64648438,     0.        ], dtype=float32)

In [33]:
reform_simulation.calculate('cotisations_contributives')

array([-4545.390625, -4545.390625,     0.      ], dtype=float32)

In [34]:
reform_simulation.calculate('salaire_de_base')

array([ 9000.,  6000.,     0.], dtype=float32)

In [35]:
reform_simulation.calculate_add('nouv_salbrut')

array([ 16543.02734375,  13543.02832031,      0.        ], dtype=float32)

In [36]:
reform_simulation.calculate_add('salaire_net')

array([ 5855.69482422,  3091.48974609,     0.        ], dtype=float32)